In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import datetime

In [4]:
### Functions
### unchanged from http://stackoverflow.com/questions/30745587/why-does-selenium-return-the-source-of-the-previously-loaded-page-in-python 
def read_page(driver, url):
    driver.get(url)
    text = driver.page_source
    return text

def get_league_matches(driver, country, league, league_url):
    print "Country:", country
    print "League:", league
    print "League URL:", league_url
    print

    league_matches = []

    # Read url
    try:
        headers = {'User-agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) \
                    Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1'}
        r = requests.get(league_url, headers=headers)
        text = r.text
    except:
        pass

    if text:
        soup = BeautifulSoup(text)
        seasons_details = []
        for i in soup.findAll("div", {"class": "main-menu2 main-menu-gray"})[0].findAll("li"):
            season_url = "".join([site_url, i.span.a["href"]])
            season = int(i.text.split("/")[-2])
            seasons_details.append([season, season_url])            
        seasons_details = sorted(seasons_details)   

        seasons_details = seasons_details[-1:]
        for each_season in seasons_details:
            season, seasons_url = each_season
            print "Season:", season
            print "Season URL:", seasons_url
            season_matches = get_season_matches(driver, season, country, league, seasons_url)
            if season_matches:
                league_matches.extend(season_matches)
                print
        return league_matches
    else:
        return None


def get_season_matches(driver, season, country, league, seasons_url):
    season_matches = []
    try:
        text = read_page(driver, seasons_url)
    except Exception, e:
            print "Season matches error:", e
            print ">>", seasons_url
    print "Read season url:", len(text)
    if text:
            try:
                page_numbers = list(set(re.findall('href="#/page/([0-9]{1})/"', text)))
            except Exception, e:
                page_numbers = ""
            if page_numbers:
                max_page_num = max(map(int, page_numbers))
                pages_urls = map(lambda x: "".join([seasons_url, "#/page/", str(x), "/"]), range(2, max_page_num+2)) # max_page_num+2))

                for pages_url in pages_urls:
                    page_matches = get_page_matches(driver, season, country, league, pages_url)
                    season_matches.extend(page_matches)
                return season_matches
            else:
                return None
    else:
        return None


def get_page_matches(driver, season, country, league, pages_url):
    page_matches = []
    try:
        text = read_page(driver, pages_url)
    except Exception, e:
        print "Get page matches error:", e
    if text:
        try:
            odds_type = re.findall('id="user-header-oddsformat-expander"><span>(.*?)</span>', text)
            if odds_type:
                odds_type = odds_type[0]
            else:
                odds_type = ""
        except Exception, e:
            odds_type = ""
        
        soup = BeautifulSoup(text)
        for i in soup.findAll("tr", {"class":re.compile("deactivate")}):
            
            try:
                timestamp = int(re.findall("[0-9]{10}", str(i.get_text))[0])
                datetime_date = datetime.datetime.utcfromtimestamp(timestamp)
                match_date = datetime_date.strftime('%d-%m-%Y')
                match_time = datetime_date.strftime('%H:%M')
            except Exception, e:
                timestamp, datetime_date, match_date, match_time = "", "", "", ""     

            try:
                ftsc = i.findAll("td", {"class": re.compile("table-score")})[0].text
                fthg, ftag = map(int, ftsc.split(":"))
                ftr_dict = {1: "H", 0: "D", -1: "A"}
                ftr = ftr_dict[cmp(fthg, ftag)]
            except Exception, e:
                ftsc, fthg, ftag, ftr = "", "", "", ""

            try:
                avoh, avod, avoa = re.findall('odds_text">(.*?)</a>', str(i.get_text))
            except Exception, e:
                avoh, avod, avoa = "", "", ""

            try:
                match_url = i.findAll("td", {"class": "name table-participant"})[0]
                hometeam, awayteam = map(lambda x: x.strip(), match_url.text.split("-"))
                match_url = "".join([site_url, match_url.a["href"]])
            except Exception, e:
                match_url, hometeam, awayteam = "", "", ""

            print ">>", match_date, match_time, hometeam, awayteam, ftsc, fthg, ftag, avoh, avod, avoa

            match_details = {"SEASON": season, "COUNTRY": country, "LEAGUE": league, "DATE": match_date, 
                             "TIME": match_time, "HOMETEAM": hometeam, "AWAYTEAM": awayteam, 
                             "FTSC": ftsc, "FTHG": fthg, "FTAG": ftag, "FTR": ftr, "AVOH": avoh, "AVOD": avod, 
                             "AVOA": avoa, "MATCH_URL": match_url, "TIMESTAMP": timestamp, "ODDS_TYPE": odds_type,
                             "PAGE_URL": pages_url}
            page_matches.append(match_details)
        return page_matches
    else:
        return None, None

In [5]:
# Call
site_url = "http://www.oddsportal.com"
soccer_url = "http://www.oddsportal.com/results/#soccer" 

driver = webdriver.PhantomJS()
country = "Germany"
league = "Bundesliga"
league_url = "http://www.oddsportal.com/soccer/germany/bundesliga/results/"
league_matches = get_league_matches(driver, country, league, league_url)
print len(league_matches)

Country: Germany
League: Bundesliga
League URL: http://www.oddsportal.com/soccer/germany/bundesliga/results/

Season: 2015
Season URL: http://www.oddsportal.com/soccer/germany/bundesliga/results/
Read season url: 267373
>> 02-03-2016 19:00 B. Monchengladbach VfB Stuttgart 4:0 4 0 2.13 3.84 3.19
>> 02-03-2016 19:00 Bayer Leverkusen Werder Bremen 1:4 1 4 1.57 4.39 5.73
>> 02-03-2016 19:00 Bayern Munich Mainz 1:2 1 2 1.14 8.68 20.60
>> 02-03-2016 19:00 Darmstadt Dortmund 0:2 0 2 6.51 4.11 1.55
>> 02-03-2016 19:00 Hertha Berlin Eintracht Frankfurt 2:0 2 0 1.95 3.46 4.08
>> 02-03-2016 19:00 Hoffenheim Augsburg 2:1 2 1 2.24 3.54 3.17
>> 02-03-2016 19:00 Schalke Hamburger SV 3:2 3 2 1.87 3.57 4.40
>> 01-03-2016 19:00 Hannover Wolfsburg 0:4 0 4 4.08 3.61 1.91
>> 01-03-2016 19:00 Ingolstadt 1. FC Koln 1:1 1 1 2.58 3.04 3.05
>> 28-02-2016 18:30 Eintracht Frankfurt Schalke 0:0 0 0 2.64 3.44 2.66
>> 28-02-2016 16:30 Dortmund Hoffenheim 3:1 3 1 1.26 6.37 11.82
>> 28-02-2016 16:30 Mainz Bayer Leverk

/home/pi/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [21]:
def get_next_matches(driver, season, country, league, pages_url):
    page_matches = []
    try:
        text = read_page(driver, pages_url)
    except Exception, e:
        print "Get page matches error:", e
    if text:
        try:
            odds_type = re.findall('id="user-header-oddsformat-expander"><span>(.*?)</span>', text)
            if odds_type:
                odds_type = odds_type[0]
            else:
                odds_type = ""
        except Exception, e:
            odds_type = ""
        
        soup = BeautifulSoup(text)
#        print soup.findAll("tr")
        for i in soup.findAll("tr"):
            
            try:
                timestamp = int(re.findall("[0-9]{10}", str(i.get_text))[0])
                datetime_date = datetime.datetime.utcfromtimestamp(timestamp)
                match_date = datetime_date.strftime('%d-%m-%Y')
                match_time = datetime_date.strftime('%H:%M')
            except Exception, e:
                timestamp, datetime_date, match_date, match_time = "", "", "", ""     

            try:
                avoh, avod, avoa = re.findall('odds_text">(.*?)</a>', str(i.get_text))
            except Exception, e:
                avoh, avod, avoa = "", "", ""

            try:
                match_url = re.findall('</script>(.*?)</td>', str(i.get_text))
                opponents = re.search('(?<=>)[^}]*(?=</a>)', match_url[0]).group(0)
                opponents_split = opponents.split("-")
                #print opponents_split
                hometeam = opponents_split[0]
                awayteam = opponents_split[1]
                #print hometeam, awayteam
                #match_url = "".join([site_url, match_url.a["href"]])
            except Exception, e:
                match_url, hometeam, awayteam = "", "", ""
                timestamp, datetime_date, match_date, match_time = "", "", "", ""     

            print ">>", match_date, match_time, hometeam, awayteam, avoh, avod, avoa

            match_details = {"SEASON": season, "COUNTRY": country, "LEAGUE": league, "DATE": match_date, 
                             "TIME": match_time, "HOMETEAM": hometeam, "AWAYTEAM": awayteam, 
                             "AVOH": avoh, "AVOD": avod, 
                             "AVOA": avoa, "MATCH_URL": match_url, "TIMESTAMP": timestamp, "ODDS_TYPE": odds_type,
                             "PAGE_URL": pages_url}
            if match_details['AVOH'] <> "":
                page_matches.append(match_details)
        return page_matches
    else:
        return None, None

In [22]:
# Call
site_url = "http://www.oddsportal.com"
soccer_url = "http://www.oddsportal.com/results/#soccer" 

driver = webdriver.PhantomJS()
season = 2015
country = "Germany"
league = "Bundesliga"
league_url = "http://www.oddsportal.com/soccer/germany/bundesliga/"
next_matches = get_next_matches(driver, season, country, league, league_url)
print len(league_matches)

>>       
>>       
>>       
>> 05-03-2016 14:30 Augsburg   Bayer Leverkusen 2.90 3.31 2.51
>> 05-03-2016 14:30 Eintracht Frankfurt   Ingolstadt 2.57 3.18 2.94
>> 05-03-2016 14:30 1. FC Koln   Schalke 2.50 3.31 2.95
>> 05-03-2016 14:30 VfB Stuttgart   Hoffenheim 1.90 3.87 3.86
>> 05-03-2016 14:30 Werder Bremen   Hannover 1.74 3.90 4.68
>> 05-03-2016 14:30 Wolfsburg   B. Monchengladbach 2.02 3.71 3.60
>> 05-03-2016 17:30 Dortmund   Bayern Munich 3.19 3.49 2.27
>>       
>>       
>> 06-03-2016 14:30 Mainz   Darmstadt 1.78 3.61 4.92
>> 06-03-2016 16:30 Hamburger SV   Hertha Berlin 2.89 3.17 2.62
>>       
>>       
>> 11-03-2016 19:30 Hertha Berlin   Schalke 2.37 3.23 3.11
>>       
>>       
>> 12-03-2016 14:30 B. Monchengladbach   Eintracht Frankfurt 1.64 4.12 4.91
>> 12-03-2016 14:30 Darmstadt   Augsburg 2.56 3.17 2.87
>> 12-03-2016 14:30 Hannover   1. FC Koln 3.19 3.26 2.31
>> 12-03-2016 14:30 Hoffenheim   Wolfsburg 2.92 3.49 2.36
>> 12-03-2016 14:30 Ingolstadt   VfB Stuttgart 2.63 

In [6]:
# Export historic data as CSV

import csv

path = '../data/match_odds.csv'
keys = league_matches[0].keys()
with open(path, 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(league_matches)
print "Stored in", path

Stored in ../data/match_odds.csv


In [27]:
# Export next matches data as CSV

import csv

path = '../data/next_match_odds.csv'
keys = next_matches[0].keys()
with open(path, 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(next_matches)
print "Stored in", path

Stored in ../data/next_match_odds.csv


In [25]:
next_matches

[{'AVOA': '2.51',
  'AVOD': '3.31',
  'AVOH': '2.90',
  'AWAYTEAM': ' Bayer Leverkusen',
  'COUNTRY': 'Germany',
  'DATE': '05-03-2016',
  'HOMETEAM': 'Augsburg ',
  'LEAGUE': 'Bundesliga',
  'MATCH_URL': ['<a href="/soccer/germany/bundesliga/augsburg-bayer-leverkusen-bgTykhNd/">Augsburg - Bayer Leverkusen</a>'],
  'ODDS_TYPE': u'EU Odds',
  'PAGE_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/',
  'SEASON': 2015,
  'TIME': '14:30',
  'TIMESTAMP': 1457188200},
 {'AVOA': '2.94',
  'AVOD': '3.18',
  'AVOH': '2.57',
  'AWAYTEAM': ' Ingolstadt',
  'COUNTRY': 'Germany',
  'DATE': '05-03-2016',
  'HOMETEAM': 'Eintracht Frankfurt ',
  'LEAGUE': 'Bundesliga',
  'MATCH_URL': ['<a href="/soccer/germany/bundesliga/eintracht-frankfurt-ingolstadt-fLMpmWi9/">Eintracht Frankfurt - Ingolstadt</a>'],
  'ODDS_TYPE': u'EU Odds',
  'PAGE_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/',
  'SEASON': 2015,
  'TIME': '14:30',
  'TIMESTAMP': 1457188200},
 {'AVOA': '2.95',
  'AVOD': '3.